# Lesson 06 文本抄袭自动检测

In [1]:
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

## 数据预处理

In [2]:
# 加载停用词
with open("./chinese_stopwords.txt", "r", encoding="utf-8") as f:
    stopwords = [word[:-1] for word in f.readlines()]
    
# 数据加载
news = pd.read_csv("./sqlResult.csv", encoding="gb18030")
news.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
# 更新数据 只需要source， content
df = news[["source","content"]]

# 处理缺省值
df.dropna(subset=["content"],inplace=True)

df.index = range(df.shape[0])
df

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,source,content
0,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...
...,...,...
87049,新华社,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...
87050,新华社,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...
87051,新华社,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...
87052,新华社,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...


In [4]:
# 分词
def text_split(text):
    text = text.replace(" ", "")
    text = text.replace("\r", "")
    text = text.replace("\n", "")
    res = jieba.cut(text)
    res = " ".join([word for word in res if word not in stopwords])
    return res

import pickle, os
path = "./corpus.pkl"
if os.path.exists(path):
    with open(path, "rb") as file:
        corpus = pickle.load(file)
else:
    corpus = list(map(text_split, [str(i) for i in news.content]))
    with open(path, "wb") as file:
        pickle.dump(corpus, file)
        
# 添加corpus特征列
df["corpus"] = corpus
# df

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# 计算corpus的TFIDF矩阵
tfidf_vec = TfidfVectorizer(encoding='gb18030', min_df=0.015) 
tfidf_matrix = tfidf_vec.fit_transform(corpus)

In [6]:
# kmeans 聚类降维
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# 规范化到[0,1]空间
min_max_scaler=MinMaxScaler()
tfidf = min_max_scaler.fit_transform(tfidf_matrix.toarray())

kmeans = KMeans(n_clusters=25)

path = "./k_label.pkl"
if os.path.exists(path):
    with open(path, "rb") as file:
        k_label = pickle.load(file)
else:
    k_label = kmeans.fit_predict(tfidf)
    with open(path, "wb") as file:
        pickle.dump(k_label, file)

In [7]:
# 添加k_label特征列
df["k_label"] = k_label

# 添加特征列  1 新华社文章   0 非新华社文章
df["label"] = list(map(lambda source:1 if "新华" in str(source) else 0, df.source))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## 贝叶斯模型

In [8]:
# 训练并预测可疑文章
from sklearn.naive_bayes import MultinomialNB
x = tfidf_matrix.toarray()
y = df.label
clf = MultinomialNB()
clf.fit(x, y)

# 预测非新华社文章
y_pred = clf.predict(x)
# 添加预测label列
df["label_pred"] = y_pred
df

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,source,content,corpus,k_label,label,label_pred
0,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测...,14,0,0
1,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 ...,3,0,0
2,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A ...,14,0,0
3,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,这是 18 葡萄牙 中部 佩德罗 冈 地区 拍摄 森林 大火 烧毁 汽车 新华社 记者 张立 云摄,4,1,1
4,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … @ 深圳 交警...,14,0,1
...,...,...,...,...,...,...
87049,新华社,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,新华社 照片 多伦多 2017 \ n 体育 冰球 国家 女子 冰球队 海外 选秀 多伦多 ...,4,1,1
87050,新华社,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,新华社 兰州 日电 王衡 徐丹 记者 甘肃省 交通运输 厅 获悉 甘肃 近日 集中 开建 高...,14,1,0
87051,新华社,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,\ n \ n2017 29 法国巴黎 郊外 凡尔赛宫 法国 总统 马克 龙 出席 新闻 发...,20,1,1
87052,新华社,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,\ n \ n2017 25 美国 马萨诸塞州 剑桥市 哈佛大学 毕业生 毕业典礼 欢呼 新...,3,1,1


In [9]:
# 可疑文章
df[(df.label==0) & (df.label_pred==1)]

,source,content,corpus,k_label,label,label_pred
4,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … @ 深圳 交警...,14,0,1
24,凤凰体育,北京时间6月20日，江苏苏宁足球俱乐部发布了足协杯第四轮江苏苏宁易购队主场对阵河南建业队的官...,北京 时间 江苏 苏宁 足球 俱乐部 发布 足协杯 第四轮 江苏 苏宁 易 购队 主场 对阵...,14,0,1
28,中超球评,中超联赛第13轮比赛已经全部结束，这轮比赛中最让人觉得结果有些出乎意料的，莫过于卡佩罗执教江...,中超联赛 13 轮 比赛 已经 全部 结束 这轮 比赛 最让人 觉得 出乎意料 莫过于 卡佩...,18,0,1
30,人民日报,“我刚刚启动发动机，就听见清脆的玻璃破裂声，然后一匹骏马出现在副驾驶座位上，含情脉脉地看着我...,刚刚 启动 发动机 听见 清脆 玻璃 破裂声 一匹 骏马 出现 副驾驶 座位 含情脉脉 看着...,14,0,1
32,快科技@http://www.kkj.cn/,其他规格方面，搭载飞思卡尔IMX6SL处理器，内存512MB，存储8GB，采用定制版系统，低...,规格 方面 搭载 飞思 卡尔 IMX6SL 处理器 内存 512MB 存储 8GB 采用 定...,14,0,1
...,...,...,...,...,...,...
8548,郑州日报第01版,本报讯（记者 聂春洁）记者昨日从铁路部门获悉，全国铁路图7月1日将迎来大调整，届时备受关注的...,本报讯 记者 聂春洁 记者 昨日 铁路 部门 获悉 全国 铁路 图 迎来 调整 届时 备受 ...,14,0,1
8549,cnBeta.COM,传奇收藏版包括：73厘米高的Bayek与Senu（宠物鹰）雕像、限量编号证书（编号从1至99...,传奇 收藏版 包括 73 厘米 Bayek Senu 宠物 鹰 雕像 限量 编号 证书 编号...,14,0,1
8551,郑州日报第01版,郑报融媒记者 张立 石闯 顾翔 文/图\r\n【脱贫攻坚·记者蹲点】\r\n夏日里，乱石坡房...,郑报 融媒 记者 张立石 闯 顾翔文 图 【 脱贫 攻坚 记者 蹲点 】 夏日 乱石 坡 房...,0,0,1
8552,长江日报第1版,????新华社电?经中央军委主席习近平批准，我军新设立“八一勋章”，并组织开展首次评选。新设...,? ? ? ? 新华社 电 ? 中央军委 主席 习近平 批准 我军 设立 八一 勋章 组织 ...,1,0,1


## 余弦相似度判别相似文本

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import heapq

# 相似文本 （top 3）

def similar_text(cpindex, topN = 3):
    local_k_label = df.iloc[cpindex].k_label     # 样本聚类标签
    local_corpus = df.iloc[cpindex].corpus       # 样本corpus
    
    # 该 k_label 下所有可疑文章
    val_df = df[(df["k_label"]==local_k_label)&(df["label"]==0)&(df["label_pred"]==1)]
    val_corpus = val_df.corpus
    
    
    local_tfidf = tfidf_vec.transform([local_corpus])
    val_tfidf = tfidf_vec.transform(val_corpus)
    
    # 余弦相似度
    res = cosine_similarity(local_tfidf, val_tfidf)
    res = list(res[0])
    
    res1 = list(map(res.index, heapq.nlargest(topN, res)))      # 求最大的三个索引 
    res2 = heapq.nlargest(topN, res)                            # 求最大的三个元素
    
    return res1, res2
    

## 验证

In [11]:
cpindex = 3134
pred_index, _ = similar_text(cpindex)

# 原文
df.iloc[cpindex]

source                                                      新华社
content       　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回...
corpus        国家统计局 19 发布 数据 月份 一线 热点 二线 城市 新建 商品住宅 价格 同比 涨幅...
k_label                                                      17
label                                                         1
label_pred                                                    1
Name: 3134, dtype: object

In [12]:
# 相似文章
for i in range(len(pred_index)):
    print(f"相似文章 {i} ：")
    print(df[(df["k_label"]==df.iloc[cpindex].k_label)&(df["label"]==0)&(df["label_pred"]==1)].iloc[pred_index[i]], "\n")

相似文章 0 ：
source                                                    国家统计局
content       　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点...
corpus        中国 月份 56 座 城市 新建 商品住宅 价格 环比 上涨 月份 58 座 上涨 月份 一...
k_label                                                      17
label                                                         0
label_pred                                                    1
Name: 3352, dtype: object 

相似文章 1 ：
source                                                     证券日报
content       　　从环比看，9个城市新建商品住宅价格下降或持平\r\n　　6月19日，国家统计局发布的20...
corpus        环比 看 城市 新建 商品住宅 价格下降 持平 19 国家统计局 发布 2017 月份 70...
k_label                                                      17
label                                                         0
label_pred                                                    1
Name: 3125, dtype: object 

相似文章 2 ：
source                                                     投资快报
content       　　国家统计局周一发布了2017年5月份70个大中城市住宅销售价格统计数据。其中，北京上海新...
corpus        国家统计局 周